# Notebook to create the Cross-Validation-Setup & train a first model

In [32]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
from glob import glob
import os
from tqdm.notebook import tqdm
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torchinfo import summary
from copy import deepcopy
import segmentation_models_pytorch as smp
from torchinfo import summary
import random
from copy import deepcopy
import albumentations as A

In [2]:
folder = '/home/olli/Projects/Kaggle/Vesuvius'

In [3]:
folder_data = os.path.join(folder, 'Data', 'Preprocessed', 'Cropped_Regions')

In [4]:
files = glob(folder_data + '/*.pickle')

In [5]:
len(files)

137

In [6]:
#sort them in the correct order to later get the same results
files.sort()

# 1) Create 5 folds from the 137 cropped parts

In [7]:
def create_folds(files):
    
    files_copy = deepcopy(files)
    
    # shuffle them with the same random seed to get identical results
    random.Random(42).shuffle(files_copy)
    
    # each fold will consist of 27, 27, 27, 28, 28 files
    num_per_fold = [27, 27, 27, 28, 28]
    folds = [f'fold_{i}' for i in range(5)]
    
    for num, fold in zip(num_per_fold, folds):
        globals()[fold] = files_copy[:num]  # assign the first num elements to the current fold
        del files_copy[:num]  # now remove them from the list
        
    folds = [fold_0, fold_1, fold_2, fold_3, fold_4]
    
    if len(files_copy) == 0:
        print('All scans assinged to their folds')
    
    return folds

In [8]:
folds = create_folds(files=files)

All scans assinged to their folds


In [9]:
[len(i) for i in folds]

[27, 27, 27, 28, 28]

In [10]:
folds[0][0]

'/home/olli/Projects/Kaggle/Vesuvius/Data/Preprocessed/Cropped_Regions/1_5.pickle'

# 2) From these 5 folds create the 5 train and valid folds

In [11]:
def create_train_valid(folds):
    
    train_folds = [f'train_{i}' for i in range(5)]
    valid_folds = [f'valid_{i}' for i in range(5)]
    
    # each time one unique fold is the validation-data and the rest is for training
    for i in range(5):
        folds_copy = deepcopy(folds)
        
        globals()[valid_folds[i]] = folds_copy.pop(i)  # current for for validation
        
        train = []  # append the 4 remaining 4 folds to the current train data
        for fold in folds_copy:
            train += fold
            
        # finally assing it to the variable
        globals()[train_folds[i]] = train
        
    train_data = [train_0, train_1, train_2, train_3, train_4]
    valid_data = [valid_0, valid_1, valid_2, valid_3, valid_4]
    
    return train_data, valid_data

In [12]:
train_data, valid_data = create_train_valid(folds=folds)

In [13]:
[len(i) for i in train_data]

[110, 110, 110, 109, 109]

In [14]:
[len(i) for i in valid_data]

[27, 27, 27, 28, 28]

In [15]:
# make sure there is no validation data in the corresponding train data

for i in range(5):
    val = valid_data[i]
    train = train_data[i]
    
    for file in val:
        if file in train:
            print(f'DUBLICATE: {file}')

# 3) Now create a model a simple 2D CNN for this segmentation task

In [16]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [17]:
model = smp.Unet(
    encoder_name='se_resnext50_32x4d',        # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
    encoder_weights=None,     # use `imagenet` pre-trained weights for encoder initialization
    in_channels=5,                  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
    classes=1,                      # model output channels (number of classes in your dataset)
)

In [18]:
summary(model, input_data=torch.randn(1, 5, 1024 , 1024))

Layer (type:depth-idx)                             Output Shape              Param #
Unet                                               [1, 1, 1024, 1024]        --
├─SENetEncoder: 1-1                                [1, 5, 1024, 1024]        --
│    └─Sequential: 2-1                             --                        --
│    │    └─Conv2d: 3-1                            [1, 64, 512, 512]         15,680
│    │    └─BatchNorm2d: 3-2                       [1, 64, 512, 512]         128
│    │    └─ReLU: 3-3                              [1, 64, 512, 512]         --
│    │    └─MaxPool2d: 3-4                         [1, 64, 256, 256]         --
│    └─Sequential: 2-2                             [1, 256, 256, 256]        --
│    │    └─SEResNeXtBottleneck: 3-5               [1, 256, 256, 256]        71,952
│    │    └─SEResNeXtBottleneck: 3-6               [1, 256, 256, 256]        79,632
│    │    └─SEResNeXtBottleneck: 3-7               [1, 256, 256, 256]        79,632
│    └─Sequential:

In [19]:
model.decoder

UnetDecoder(
  (center): Identity()
  (blocks): ModuleList(
    (0): DecoderBlock(
      (conv1): Conv2dReLU(
        (0): Conv2d(3072, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
      )
      (attention1): Attention(
        (attention): Identity()
      )
      (conv2): Conv2dReLU(
        (0): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
      )
      (attention2): Attention(
        (attention): Identity()
      )
    )
    (1): DecoderBlock(
      (conv1): Conv2dReLU(
        (0): Conv2d(768, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inpla

### The model has no sigmoid activation as output so use BCELossWithLogits

In [20]:
model(torch.randn(1, 5, 1024, 1024).to(device)).size()

torch.Size([1, 1, 1024, 1024])

### Now save the initial weights to start a new model by loading these

In [23]:
def save_weights(model):
    name = 'UNET_random_weights.pth'
    
    path_weight = os.path.join(folder, 'Weights', name)
    
    if not os.path.exists(path_weight):
        
        torch.save(model.state_dict(), path_weight)

In [24]:
save_weights(model=model)

# 4) Now define the functions to train and evaluate a model

In [25]:
def train_model(model, optimizer, loss, X, y):
    model.train()
    
    pred = model(X)
    
    batch_loss = loss(pred, y)
    
    batch_loss.backward()
    
    optimizer.step()
    
    optimizer.zero_grad()
    
    return batch_loss.item()

In [26]:
@torch.no_grad()
def valid_loss(model, optimizer, loss, X, y):
    model.eval()
    
    pred = model(X)
    
    batch_loss = loss(pred, y)
    
    return batch_loss.item()

In [27]:
def calculate_dice(pred, y, beta=0.5, smooth=1e-5):

    pred = torch.sigmoid(pred)  # model has no sigmoid

    # create a single dimension float vector
    pred = pred.view(-1).float()
    y = y.view(-1).float()

    y_true_count = y.sum()
    ctp = pred[y==1].sum()
    cfp = pred[y==0].sum()
    beta_squared = beta * beta

    c_precision = ctp / (ctp + cfp + smooth)
    c_recall = ctp / (y_true_count + smooth)
    dice = (1 + beta_squared) * (c_precision * c_recall) / (beta_squared * c_precision + c_recall + smooth)

    return dice_score

In [28]:
@torch.no_grad()
def eval_dice(model, X, y):
    model.eval()
    
    pred = model(X)
    
    dice_score = calculate_dice(pred, y)
    
    return dice_score

# 5) Define the augmentations (use albumentations with mask!)

In [29]:
# diceloss

In [36]:
augmentations_train = A.Compose([
    A.RandomResizedCrop(height=1024, width=1024, scale=(0.75, 1)),
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.Rotate(limit=20, p=0.8),
    A.GaussNoise(p=0.8),
    A.ElasticTransform(p=0.8),
    A.Normalize(mean=[0.5] * 5, std=[0.5] * 5)  # value for each layer
])

# 6) Dataset and DataLoader

In [30]:
# gets initialized with the paths from the corresponding five train/valid datasets

class Data(Dataset):
    def __init__(self, paths, transform=False):
    
        self.paths = paths
        random.shuffle(self.paths)
        
        self.transform = transform
        
    def __len__(self):
        return len(self.paths)
    
    def __getitem__(self, index):
        path = self.paths[index]
        
        with open(path, 'rb') as f:
            data = pickle.load(f)
            
        X, y = data
        
        y = y / 255.
        y = torch.tensor(y).float()
        
        X = X[:5]  # 5 layers as defined
        
        return X, y